# HMM Exercise

In this exercise, we will use the ``hmmlearn`` library to practice what we have learned about Hidden Markov Models (HMMs), both on a toy problem and to build a POS tagger from scratch. First make sure that ``hmmlearn`` is installed:

In [59]:
!pip install hmmlearn

## Part 1: Seasonal weather

In this section, we will consider the following toy problem:

A year has four seasons - winter, spring, summer, and fall. Every day of the year we go outside, look at the sky to see if it is sunny or rainy, and try to guess what season it is.

**Questions:**
1. If we model this with an HMM, what are the hidden states? What are the observed variables?

Hidden states represent the seasons (Winter, Spring, Summer, Fall).
Observed variables represent the weather conditions (Sunny, Rainy).

2. Using ``hmmlearn.hmm.CategoricalHMM``, create an HMM model for this problem. Note: you may refer to [the hmmlearn API documentation](https://hmmlearn.readthedocs.io/en/latest/api.html).
  
  * Instantiate the HMM model with ``weather_model = CategoricalHMM(n_components=...)``, where ``n_components`` is set to the number of hidden states.
  </br>
  
  * Set ``weather_model.startprob_`` to be uniform (all hidden states with equal probability)
      </br>Hint: ``weather_model.startprob_`` should be a list of numbers.
  </br>
  
  * Set ``weather_model.transmat_`` so that on every day, there is a 0.99 probability of the season remaining the same as the previous day and a 0.01 probability of it being the next season.
      </br>Hint: ``model.transmat_`` should be a NumPy array of shape ``(n_components, n_components)``
  </br>
  * Set ``weather_model.emissionprob_`` so that 90% of summer days are sunny, 90% of winter days are rainy, and 50% of spring and fall days are sunny or rainy.
      </br>Hint: ``model.emissionprob_`` should be a NumPy array of shape ``(n_components, n_features)``.
  

In [60]:
from hmmlearn import hmm
import numpy as np

In [61]:
n_states = 4
weather_model = hmm.CategoricalHMM(n_components=n_states)

weather_model.startprob_ = np.ones(n_states) / n_states

transition_prob_same_season = 0.99
transition_prob_next_season = 0.01

transmat = np.zeros((n_states, n_states))
for i in range(n_states):
    transmat[i, i] = transition_prob_same_season
    transmat[i, (i + 1) % n_states] = transition_prob_next_season

weather_model.transmat_ = transmat

emissionprob = np.array([[0.1, 0.9], # Summer
                         [0.5, 0.5], # Fall
                         [0.9, 0.1], # Winter
                         [0.5, 0.5]]) # Spring

weather_model.emissionprob_ = emissionprob


3. Sample 100 days from the model by using  ``weather_model.sample(100)``. How many sunny days are observed? How many times did the season change?

In [62]:
weather_samples, sampled_states  = weather_model.sample(100)

num_sunny_days = np.sum(weather_samples == 1)

num_season_changes = 0
for i in range(len(sampled_states) - 1) :
    if sampled_states[i] != sampled_states[i+1]:
        num_season_changes += 1

print("Number of sunny days observed:", num_sunny_days)
print("Number of times the season changed:", num_season_changes)

Number of sunny days observed: 55
Number of times the season changed: 0


4. If 50 sunny days and then 50 rainy days are observed, what is the most likely sequence of seasons for those days under this model? Use ``weather_model.decode(...)`` to determine this.
</br>Hint: The input to this function should be a NumPy array of shape ``(100, 1)``.

In [63]:
observed_sequence = np.concatenate((np.ones((50, 1), dtype=int), np.zeros((50, 1), dtype=int)), axis=0)
_ , predicted_states = weather_model.decode(observed_sequence)

print(predicted_states)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


## Part 2: Building a POS tagger

In this section, we will build a simple Part-of-Speech (POS) tagger for English texts based on labelled data from the [Brown Corpus](https://en.wikipedia.org/wiki/Brown_Corpus). First make sure that the necessary corpora are downloaded:

In [64]:
import nltk
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

Now we will build a HMM-based POS tagger from scratch, step-by-step:

**Questions:**

5. Get POS-tagged sentences from the Brown corpus using ``nltk.corpus.brown.tagged_sents(tagset='universal')``. Use ``sklearn.model_selection.train_test_split`` to split them into 80% training data and 20% testing data.

In [65]:
from sklearn.model_selection import train_test_split

tagged_sentences = nltk.corpus.brown.tagged_sents(tagset='universal')

train_data, test_data = train_test_split(tagged_sentences, test_size=0.2, random_state=42)

print("Number of training sentences:", len(train_data))
print("Number of testing sentences:", len(test_data))


Number of training sentences: 45872
Number of testing sentences: 11468


In [66]:
print("Sample sentences from the training data:")
for i in range(5):
    print(train_data[i])

Sample sentences from the training data:
[('frontiers', 'NOUN'), ('.', '.')]
[('The', 'DET'), ('pastor', 'NOUN'), ('calls', 'VERB'), ('in', 'ADP'), ('the', 'DET'), ('home', 'NOUN'), ('of', 'ADP'), ('each', 'DET'), ('individual', 'NOUN'), ('or', 'CONJ'), ('family', 'NOUN'), ('for', 'ADP'), ('a', 'DET'), ('spiritual', 'ADJ'), ('guidance', 'NOUN'), ('conference', 'NOUN'), ('.', '.')]
[('``', '.'), ('Standard', 'ADJ'), ('deal', 'NOUN'), (',', '.'), ('Mr.', 'NOUN'), ('Skyros', 'NOUN'), ('.', '.')]
[('Mr.', 'NOUN'), ('Mills', 'NOUN'), ('had', 'VERB'), ('done', 'VERB'), ('some', 'DET'), ('figuring', 'NOUN'), ('on', 'ADP'), ('a', 'DET'), ('scrap', 'NOUN'), ('of', 'ADP'), ('paper', 'NOUN'), ('and', 'CONJ'), ('given', 'VERB'), ('him', 'PRON'), ('the', 'DET'), ('various', 'ADJ'), ('kinds', 'NOUN'), ('of', 'ADP'), ('boards', 'NOUN'), ('and', 'CONJ'), ('two-by-fours', 'NOUN'), ('which', 'DET'), (',', '.'), ('properly', 'ADV'), ('handled', 'VERB'), (',', '.'), ('would', 'VERB'), (',', '.'), ('he', '

6. Define ``POS_TAGS`` to be an array of all unique POS tags found in the training data, and define `N_TAGS` to be the number of unique POS tags in this array. What is the value of `N_TAGS`?

In [67]:
POS_TAGS = set(tag for sentence in train_data for word, tag in sentence)
N_TAGS = len(POS_TAGS)

print("Number of unique POS tags:", N_TAGS)
print(POS_TAGS)

Number of unique POS tags: 12
{'NOUN', 'DET', 'PRT', 'ADV', 'PRON', 'ADJ', 'X', '.', 'VERB', 'NUM', 'CONJ', 'ADP'}


7. Using `collections.Counter`, find the 5000 most common word tokens in the training data, and save their unique values to an array ``vocab``. Make all words lowercase before counting, and in addition, add the token '[UNK]' as the first element of ``vocab`` (representing words which are "out of vocabulary"). Hint: the first five elements of ``vocab`` should be \["\[UNK\]", "the", ",", ".", "of", ...\]

In [68]:
from collections import Counter

word_tokens = [word.lower() for sentence in train_data for word, _ in sentence]
word_counts = Counter(word_tokens)
vocab = ["[UNK]"] + [word for word, _ in word_counts.most_common(4999)]

print("First five elements of vocab:", vocab[:5])

First five elements of vocab: ['[UNK]', 'the', ',', '.', 'of']


8. Create an array `S` of start probabilities for each POS tag (i.e. what fraction of sentences start with a noun? What fraction of them start with a verb? Etc.) Initialize `S = np.zeros(N_TAGS)` and then iterate through the sentences in the training set and add one to an element of `S` for each sentence. Finally set `S = S / sum(S)` to get probabilities.  
</br>*Note: This step should take less than 1 second to run (use tqdm to track its progress).*

In [69]:
S = np.zeros(N_TAGS)

for sentence in tqdm(train_data, desc="Calculating start probabilities"):
    first_tag = sentence[0][1]
    S[list(POS_TAGS).index(first_tag)] += 1

S /= np.sum(S)

print("\n\n Start probabilities:", S)

Calculating start probabilities: 100%|██████████| 45872/45872 [00:00<00:00, 283666.58it/s]



 Start probabilities: [0.14093565 0.21300576 0.03705964 0.09016393 0.16025026 0.03396407
 0.0003924  0.08918294 0.04606296 0.01676404 0.04843913 0.12377921]


9. Create a matrix `P` of transition probabilities of transitioning from one POS tag to another, estimated from the training data.  Initialize a matrix `P = np.zeros(shape=(N_TAGS, N_TAGS))`. Then iterate through each pair of words in the training dataset and add one to an element of `P` for each such pair. Finally set `P /= P.sum(axis=1)[:, None]` to get probabilities.
</br>*Note: This step should take less than 5 seconds to run (use tqdm to track its progress).*

In [70]:
from tqdm import tqdm

P = np.zeros(shape=(N_TAGS, N_TAGS))

for sentence in tqdm(train_data, desc="Calculating transition probabilities"):
    for i in range(len(sentence) - 1):
        tag1 = sentence[i][1]
        tag2 = sentence[i + 1][1]
        P[list(POS_TAGS).index(tag1), list(POS_TAGS).index(tag2)] += 1

P /= P.sum(axis=1)[:, None]

print("Transition probabilities matrix:")
print(P)
print(P.shape)

Calculating transition probabilities: 100%|██████████| 45872/45872 [00:01<00:00, 24213.62it/s]

Transition probabilities matrix:
[[1.50334759e-01 1.54993659e-02 1.79719921e-02 2.64398275e-02
  1.96855583e-02 1.27858406e-02 3.63621488e-04 2.84742897e-01
  1.58248072e-01 8.00421801e-03 6.00520888e-02 2.45871760e-01]
 [6.26169760e-01 5.93439300e-03 2.08160247e-03 1.75840630e-02
  9.85109238e-03 2.39959464e-01 1.36947531e-03 1.26083027e-02
  6.47853119e-02 9.85109238e-03 6.02569136e-04 9.20287407e-03]
 [3.67171528e-02 8.34063504e-02 1.18496266e-02 3.54654316e-02
  7.21825844e-03 1.86923687e-02 0.00000000e+00 7.68974006e-02
  6.21729879e-01 4.96516043e-03 1.18913506e-02 9.11670213e-02]
 [3.28587319e-02 7.35483871e-02 2.85428254e-02 9.71523915e-02
  4.73637375e-02 1.36618465e-01 8.89877642e-05 1.70322581e-01
  2.40200222e-01 1.33036707e-02 1.76195773e-02 1.42380423e-01]
 [8.80800081e-03 1.72352523e-02 2.41648898e-02 5.37364199e-02
  8.60493451e-03 9.44258300e-03 2.53832876e-05 1.03183064e-01
  7.07203777e-01 1.14224794e-03 1.13717129e-02 5.50817342e-02]
 [6.53532365e-01 5.97692905e-03 

10. Create a matrix `E` of emission probabilities, the probability of outputting each word in `vocab` for a given POS tag. Initialize `E = np.zeros(shape=(N_TAGS, len(vocab)))`. Then iterate through each word and POS tag in the training dataset. Make each word lowercase and replace words not in `vocab` with "\[UNK\]". For each word and POS tag, add one to some element of `E`. Finally set `E /= E.sum(axis=1)[:, None]` to get probabilities.
</br>*Note: This step should take approximately 10 seconds to run (use tqdm to track its progress).*

In [71]:
E = np.zeros(shape=(N_TAGS, len(vocab)))

for sentence in tqdm(train_data, desc="Calculating emission probabilities"):
    for word, tag in sentence:
        word = word.lower()
        if word not in vocab:
            word = "[UNK]"
        tag_index = list(POS_TAGS).index(tag)
        word_index = vocab.index(word)
        E[tag_index, word_index] += 1

E /= E.sum(axis=1)[:, None]

print("Emission probabilities matrix:")
print(E)

Calculating emission probabilities: 100%|██████████| 45872/45872 [00:40<00:00, 1138.80it/s]

Emission probabilities matrix:
[[3.02102965e-01 0.00000000e+00 0.00000000e+00 ... 6.79544796e-05
  7.24847782e-05 5.88938823e-05]
 [3.46889406e-04 5.10046100e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.62245579e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.13993146e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.81719026e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.68968430e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


11. Using ``hmmlearn.hmm.CategoricalHMM``, create an HMM model ``pos_model`` for POS tagging. It will have POS tags as hidden states and English word tokens as outputs. Use the results from previous questions as parameters:
  * Set `pos_model.startprob_` to `S`
  * Set `pos_model.transmat_` to `P`
  * Set `pos_model.emissionprob_` to `E`

In [72]:
from hmmlearn.hmm import CategoricalHMM

pos_model = CategoricalHMM(n_components=N_TAGS)

pos_model.startprob_ = S
pos_model.transmat_ = P
pos_model.emissionprob_ = E

12. Write a function `convert_text_to_tokens(text)`
    * This function receives a list of words
    * It searches for each word in the text in the vocabulary you built and convert it to the corresponding
    index in `vocab`
    * return value is a numpy array of indices (tokens)
    * *make sure to lowercase the word before searching in `vocab`*
   

In [73]:
def convert_text_to_tokens(text):
    tokens = []
    for word in text:
        word = word.lower()
        if word in vocab:
            tokens.append(vocab.index(word))
        else:
            tokens.append(vocab.index("[UNK]"))
    return np.array(tokens)

13. Write a function `get_pos(sentence)` that returns the most likely POS tags for the words in the string ``sentence``.
    * Convert the text to tokens using the function you wrote in the previous question
    * Pass the tokens to `pos_model.decode(...)` to get the most likely POS tags.
    
    *Note: `pos_model.decode(...)` expects a 2D numpy array*
    
    Apply this to a few (lowercase, no punctuation) sentences including:
    * "this is a test"
    * "tel aviv is in israel"
    * "i know how to write code"
    
    *split the sentences to words before calling your function*
    
    Do the results look reasonable to you?

In [74]:
def get_pos(sentence):
    tokens = convert_text_to_tokens(sentence.split())
    tokens = tokens.reshape(1, -1)
    _ , predicted_states = pos_model.decode(tokens)
    predicted_tags = [list(POS_TAGS)[i] for i in predicted_states]
    return predicted_tags

sentences = [
    "this is a test",
    "tel aviv is in israel",
    "i know how to write code",
    "He is a smart boy",
    "She is the most beautiful in the room"
]

for sentence in sentences:
    pos_tags = get_pos(sentence)
    print("Sentence:", sentence)
    print("POS tags:", pos_tags)
    print()


Sentence: this is a test
POS tags: ['DET', 'VERB', 'DET', 'NOUN']

Sentence: tel aviv is in israel
POS tags: ['NOUN', 'NOUN', 'VERB', 'ADP', 'NOUN']

Sentence: i know how to write code
POS tags: ['PRON', 'VERB', 'ADV', 'PRT', 'VERB', 'NOUN']

Sentence: He is a smart boy
POS tags: ['PRON', 'VERB', 'DET', 'ADJ', 'NOUN']

Sentence: She is the most beautiful in the room
POS tags: ['PRON', 'VERB', 'DET', 'ADJ', 'ADJ', 'ADP', 'DET', 'NOUN']



Yes, the results look reasonable.

In [75]:
for i in range(10):
  print(test_data[i])

[('Open', 'ADJ'), ('market', 'NOUN'), ('policy', 'NOUN')]
[('And', 'CONJ'), ('you', 'PRON'), ('think', 'VERB'), ('you', 'PRON'), ('have', 'VERB'), ('language', 'NOUN'), ('problems', 'NOUN'), ('.', '.')]
[('Mae', 'NOUN'), ('entered', 'VERB'), ('the', 'DET'), ('room', 'NOUN'), ('from', 'ADP'), ('the', 'DET'), ('hallway', 'NOUN'), ('to', 'ADP'), ('the', 'DET'), ('kitchen', 'NOUN'), ('.', '.')]
[('This', 'DET'), ('will', 'VERB'), ('permit', 'VERB'), ('you', 'PRON'), ('to', 'PRT'), ('get', 'VERB'), ('a', 'DET'), ('rough', 'ADJ'), ('estimate', 'NOUN'), ('of', 'ADP'), ('how', 'ADV'), ('much', 'ADJ'), ('the', 'DET'), ('materials', 'NOUN'), ('for', 'ADP'), ('the', 'DET'), ('shell', 'NOUN'), ('will', 'VERB'), ('cost', 'VERB'), ('.', '.')]
[('the', 'DET'), ('multifigure', 'NOUN'), ('``', '.'), ('Traveling', 'VERB'), ('Carnival', 'NOUN'), ("''", '.'), (',', '.'), ('in', 'ADP'), ('which', 'DET'), ('action', 'NOUN'), ('is', 'VERB'), ('vivified', 'VERB'), ('by', 'ADP'), ('lighting', 'VERB'), (';', '.

14. Split your test set to `X_test` (sentences) and `y_test` (tags)
    Note: Both `X_test` and `y_test` should be lists of lists

In [76]:
X_test = [[word for word, _ in sentence] for sentence in test_data]
y_test = [[tag for _, tag in sentence] for sentence in test_data]

print("Number of sentences in X_test:", len(X_test))
print("Number of tags in y_test:", len(y_test))


Number of sentences in X_test: 11468
Number of tags in y_test: 11468


In [77]:
for i in range(5):
  print(X_test[i], y_test[i])

['Open', 'market', 'policy'] ['ADJ', 'NOUN', 'NOUN']
['And', 'you', 'think', 'you', 'have', 'language', 'problems', '.'] ['CONJ', 'PRON', 'VERB', 'PRON', 'VERB', 'NOUN', 'NOUN', '.']
['Mae', 'entered', 'the', 'room', 'from', 'the', 'hallway', 'to', 'the', 'kitchen', '.'] ['NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', '.']
['This', 'will', 'permit', 'you', 'to', 'get', 'a', 'rough', 'estimate', 'of', 'how', 'much', 'the', 'materials', 'for', 'the', 'shell', 'will', 'cost', '.'] ['DET', 'VERB', 'VERB', 'PRON', 'PRT', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADV', 'ADJ', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'VERB', '.']
['the', 'multifigure', '``', 'Traveling', 'Carnival', "''", ',', 'in', 'which', 'action', 'is', 'vivified', 'by', 'lighting', ';', ';'] ['DET', 'NOUN', '.', 'VERB', 'NOUN', '.', '.', 'ADP', 'DET', 'NOUN', 'VERB', 'VERB', 'ADP', 'VERB', '.', '.']


15. Predict the POS tags of each sentence in the test set using the `get_pos` function you wrote
</br>*Note: This step should take less than 10 seconds to run (use tqdm to track its progress).*

In [78]:
predicted_tags_all = []

for sentence in tqdm(X_test, desc="Predicting POS tags"):
    sentence = " ".join(sentence)
    predicted_tags = get_pos(sentence)
    predicted_tags_all.append(predicted_tags)



Predicting POS tags: 100%|██████████| 11468/11468 [00:12<00:00, 906.61it/s] 


16. Compute the accuracy of your predictions

In [79]:
def compute_accuracy(predicted_tags_all, y_test):
    total_sentences = len(predicted_tags_all)
    correct_sentences = sum([1 for predicted_tags, true_tags in zip(predicted_tags_all, y_test) if predicted_tags == true_tags])
    accuracy = correct_sentences / total_sentences
    return accuracy

accuracy = compute_accuracy(predicted_tags_all, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.346180676665504
